In [1]:
import os, csv, time
import pandas as pd
import sparql
from collections import Counter
os.getcwd()

'/Users/mark/src/books/src'

In [2]:
# local modules
import config
import sparql

First we create an empty CSV file with the colums to fill.

## Find additional info

setup the data from the csv file to a pd dataframe and set the index of the dataframe to the titles of the books. The latter will be useful later.

In [3]:
filename = config.dataset_dir + 'bookdata_alt.csv'
info = pd.read_csv(filename)
info.index = info['title']

In [4]:
keys, _ = sparql.book_info()

In [14]:
def books_info(title_list=[], verbose=False):
    
    print('searching %i titles' % len(title_list))
    data_per_book = []
    unknown_titles = []
    current = 0
    # keys, _ = sparql.book_info()
    for title in title_list:
        current +=1
        # TODO try a couple of times
        i = 3
        success = 'pending'
        while not success == 'ok' or i == 0:
            i -= 1
            # book info returns     
            #  ('ok', (keys, [info_dict]) ) | ('error', 'http_error')
            (success, result) = sparql.book_info(title)
            if verbose: print(success, len(result[1]))
            if success == 'ok':
                _, results = result
                if not results == []:
                    # ignore all but the first result
                    # TODO save results that have an author
                    data_per_book.append(results[0])
                else: # 'http_error'
                    if verbose: print('error', title)
                    unknown_titles.append(title)

        if not success == 'ok':
            unknown_titles.append(title)
        
 
    return data_per_book, unknown_titles



Run the function books_info to get the data for the books found in dbpedia and the books that where not found in dbpedia

In [16]:
v = False
results, unknown_titles = books_info(info['title'][:1000], verbose=v)

print('n results',len(results))
print('n failed (not found)',len(unknown_titles))


searching 1000 titles
n results 121
n failed (not found) 879


Delete rows from the dataframe where the book is not found in dbpedia. Without this data, the row is useless.

In [ ]:
dropped = 0
for title in unknown_titles:
    dropped += 1
    print(dropped)
    info = info.drop(title)
    
print('dropped %i books' % dropped)

Next, we want to find the genres for all books

In [ ]:
def find_genres(dic):
    to_strip = 'http://dbpedia.org/resource/'
    genre_list = []
    title_list = []
   
    for result in dic:
        try:
            genre = (result['genre']['value'])
            genre = genre.replace(to_strip, '')
            genre = genre.replace('_', ' ')
            genre_list.append(genre)
            
        except KeyError as error:
            genre = 'unknown'
            genre_list.append(genre)
            # Output expected KeyErrors.
            continue
            
  
    # COMMENTED CODE DOES NOT WORK YET  
    genres = pd.Series(genre_list)
    info['genre'] = genres.values
    unique_genres = set(genre_list)
    genre_distribution = Counter(genre_list)
    print(len([genre for genre in genres if genre != "unknown"]))
    return unique_genres, genre_distribution, info

unique_genres, genre_distribution, info = find_genres(results)
print(genre_distribution)
info = info[info.genre != 'unknown']

info.to_csv(config.dataset_dir + '/output/final_data.csv')

create a file that stores the unique genres

In [ ]:
genres_file = open(config.dataset_dir +'output/unique_genres.txt', 'w')
for item in unique_genres:
    genres_file.write('%s\n' % item)

genres_file.close()


In [ ]:
sanitized = [sparql.sanitize(t) for t in unknown_titles]
results, unknown_titles = books_info(sanitized, verbose=v)
print('n results',len(results))
print('n failed (not found)',len(unknown_titles))

In [ ]:
# TODO string.strip
import sparql
string = "sdf ' df"
# string = string.strip('"')
# string = string.strip("'")
sparql.sanitize(string)

In [ ]:
info['title'][0:10]